In [1]:
# Do the necessary imports

import time
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline

print('setup complete')

setup complete


In [10]:
df_fe = pd.read_csv('Datasets/feature_eng.csv')
df_names = pd.read_csv('Datasets/dataset_5SecondWindow_names.csv')
df = pd.read_csv('Datasets/dataset_5SecondWindow.csv', index_col=[0])

In [39]:
df_names.describe()

,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,...,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std
count,5893.000000,5893.000000,0.0,5893.000000,5893.000000,5893.000000,5893.000000,5893.000000,5094.000000,5094.000000,...,1487.000000,232.000000,4899.000000,4899.000000,4899.000000,4062.000000,3520.000000,3520.000000,3520.000000,1318.000000
mean,10685.407263,117.731716,NaN,87.718310,10.007976,8.426262,12.247691,0.982987,0.759280,0.743872,...,10112.769334,2.959475,78.724855,78.451658,78.997213,1.913065,8.235254,8.216747,8.253528,0.236959
std,6921.229833,126.474151,NaN,24.562728,0.655949,1.783007,5.178436,1.505615,0.251174,0.255232,...,11067.568076,1.365350,14.025440,14.262912,13.889766,3.290193,11.870711,11.866102,11.877158,0.453205
min,0.000000,3.000000,NaN,3.000000,7.369055,0.337166,8.801019,0.001286,0.007770,0.007616,...,28.000000,1.414214,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4502.000000,31.000000,NaN,90.000000,9.737143,7.805295,10.015566,0.078055,0.640486,0.612155,...,1767.000000,1.414214,72.834623,72.233399,73.302235,0.034126,0.000431,0.000000,0.000431,0.000000
50%,10959.000000,70.000000,NaN,100.000000,9.850143,9.123124,10.602081,0.369163,0.840643,0.818037,...,6096.000000,3.027650,84.035672,83.859708,84.496874,0.806102,1.710000,1.700000,1.710000,0.018236
75%,17049.000000,152.000000,NaN,100.000000,10.078998,9.691731,12.291575,1.058376,0.963207,0.960858,...,12995.000000,3.593976,89.746162,89.743333,89.747576,2.253633,10.250000,10.250000,10.250000,0.252936
max,22045.000000,682.000000,NaN,100.000000,17.427391,12.115013,66.886376,12.879851,1.000000,1.000000,...,33355.000000,6.363961,90.308734,90.308734,90.308734,26.835546,51.360060,51.360060,51.360060,3.842801


In [17]:
df_fe.head()

<ipython-input-17-fcc19badef75>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_fe.columns = df_fe.columns.str.replace('android.sensor.','').str.replace('#','_')


,id,user,gyroscope_mean,accelerometer_std,gyroscope_std,gyroscope_uncalibrated_mean,accelerometer_max,linear_acceleration_mean,speed_mean,rotation_vector_mean,rotation_vector_max,accelerometer_min,magnetic_field_uncalibrated_min,sound_min,target,acc_gyro,Lin_speed
0,16170,Luca,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.180379,0.050413,0.056351,9.758895,51.199707,NaN,Still,0.000083,0.003784
1,15871,Luca,0.036326,1.775944,0.020290,0.039023,17.146631,0.879220,14.679876,0.999981,0.999999,7.707437,82.409890,89.065143,Car,0.036325,12.906844
2,16811,Luca,0.001525,0.011199,0.000713,0.016302,9.849262,0.034110,0.180379,0.610456,0.610456,9.804817,55.501802,NaN,Still,0.000931,0.006153
3,15831,Luca,0.036440,0.862553,0.010553,0.050759,12.304298,1.488361,14.679876,0.998112,0.998112,7.659674,95.664309,87.470377,Car,0.036372,21.848949
4,876,andrea,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,14.679876,0.322242,0.378193,8.965621,156.795909,89.770732,Car,0.059035,24.343749


In [19]:
df_names.columns = df_names.columns.str.replace('android.sensor.','').str.replace('#','_')
df_names.head()

<ipython-input-19-f656ceef780a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_names.columns = df_names.columns.str.replace('android.sensor.','').str.replace('#','_')


,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,...,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,Luca
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,Luca
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,andrea


In [11]:
df.head()

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,U12
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,U1


In [14]:
for name, group in df.groupby('user'):
    print(name,len(group)) 
for name, group in df_names.groupby('user'):
    print(name,len(group)) 

U1 2470
U10 531
U11 167
U12 820
U13 90
U2 271
U3 331
U4 227
U5 17
U6 279
U7 397
U8 214
U9 79
AndreaCarpineti 227
Claudio 90
Damiano 531
Elena 79
Federica 214
IvanHeibi 271
Luca 820
Pierpaolo 331
Riccardo 17
Serena 167
Vincenzo 279
andrea 2470
michelangelo 397


In [22]:
display(df_fe.head(1))
display(df_names.head(1))

,id,user,gyroscope_mean,accelerometer_std,gyroscope_std,gyroscope_uncalibrated_mean,accelerometer_max,linear_acceleration_mean,speed_mean,rotation_vector_mean,rotation_vector_max,accelerometer_min,magnetic_field_uncalibrated_min,sound_min,target,acc_gyro,Lin_speed
0,16170,Luca,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.180379,0.050413,0.056351,9.758895,51.199707,NaN,Still,0.000083,0.003784


,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,...,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.02934,0.029014,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,Still,Luca


In [25]:
selected_col = ['id','user','gyroscope_mean','accelerometer_std','gyroscope_std','gyroscope_uncalibrated_mean' ,'accelerometer_max' ,'linear_acceleration_mean' ,'speed_mean' ,'rotation_vector_mean' ,'rotation_vector_max' ,'accelerometer_min' ,'magnetic_field_uncalibrated_min' ,'sound_min' ,'target']

In [35]:
df_names_new = df_names[selected_col]
#df_names_new['rv_gyro_mean'] = df_names_new['rotation_vector_mean'] * df_names_new['gyroscope_mean']
#df_names_new['lin_speed_mean'] = df_names_new['speed_mean'] * df_names_new['linear_acceleration_mean']

In [36]:
df_names_new

,id,user,gyroscope_mean,accelerometer_std,gyroscope_std,gyroscope_uncalibrated_mean,accelerometer_max,linear_acceleration_mean,speed_mean,rotation_vector_mean,rotation_vector_max,accelerometer_min,magnetic_field_uncalibrated_min,sound_min,target
0,16170,Luca,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.000000,0.050413,0.056351,9.758895,51.199707,NaN,Still
1,15871,Luca,0.036326,1.775944,0.020290,0.039023,17.146631,0.879220,16.539349,0.999981,0.999999,7.707437,82.409890,89.065143,Car
2,16811,Luca,0.001525,0.011199,0.000713,0.016302,9.849262,0.034110,0.000000,0.610456,0.610456,9.804817,55.501802,NaN,Still
3,15831,Luca,0.036440,0.862553,0.010553,0.050759,12.304298,1.488361,17.739895,0.998112,0.998112,7.659674,95.664309,87.470377,Car
4,876,andrea,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,9.000000,0.322242,0.378193,8.965621,156.795909,89.770732,Car
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5888,14125,IvanHeibi,0.627658,3.373972,0.389385,0.476903,13.414389,4.934205,NaN,0.932682,0.941454,4.564648,56.502261,NaN,Walking
5889,19582,Pierpaolo,0.018259,0.665529,0.005964,0.018163,11.454213,0.544970,31.960054,0.295905,0.320622,8.768007,85.122017,NaN,Train
5890,16178,Luca,0.001803,0.017633,0.000904,0.016219,9.862587,0.020355,0.000000,0.051333,0.059438,9.772881,51.129444,NaN,Still
5891,10973,Damiano,0.003652,0.013485,0.001301,0.048900,9.874450,0.030764,1.090415,0.276348,0.276354,9.807347,270.328540,NaN,Still


In [37]:
df_fe

,id,user,gyroscope_mean,accelerometer_std,gyroscope_std,gyroscope_uncalibrated_mean,accelerometer_max,linear_acceleration_mean,speed_mean,rotation_vector_mean,rotation_vector_max,accelerometer_min,magnetic_field_uncalibrated_min,sound_min,target,acc_gyro,Lin_speed
0,16170,Luca,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.180379,0.050413,0.056351,9.758895,51.199707,NaN,Still,0.000083,0.003784
1,15871,Luca,0.036326,1.775944,0.020290,0.039023,17.146631,0.879220,14.679876,0.999981,0.999999,7.707437,82.409890,89.065143,Car,0.036325,12.906844
2,16811,Luca,0.001525,0.011199,0.000713,0.016302,9.849262,0.034110,0.180379,0.610456,0.610456,9.804817,55.501802,NaN,Still,0.000931,0.006153
3,15831,Luca,0.036440,0.862553,0.010553,0.050759,12.304298,1.488361,14.679876,0.998112,0.998112,7.659674,95.664309,87.470377,Car,0.036372,21.848949
4,876,andrea,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,14.679876,0.322242,0.378193,8.965621,156.795909,89.770732,Car,0.059035,24.343749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5888,14125,IvanHeibi,0.627658,3.373972,0.389385,0.476903,13.414389,4.934205,1.120206,0.932682,0.941454,4.564648,56.502261,NaN,Walking,0.585406,5.527324
5889,19582,Pierpaolo,0.018259,0.665529,0.005964,0.018163,11.454213,0.544970,34.722137,0.295905,0.320622,8.768007,85.122017,NaN,Train,0.005403,18.922512
5890,16178,Luca,0.001803,0.017633,0.000904,0.016219,9.862587,0.020355,0.180379,0.051333,0.059438,9.772881,51.129444,NaN,Still,0.000093,0.003672
5891,10973,Damiano,0.003652,0.013485,0.001301,0.048900,9.874450,0.030764,0.180379,0.276348,0.276354,9.807347,270.328540,NaN,Still,0.001009,0.005549


In [38]:
df_names_new

,id,user,gyroscope_mean,accelerometer_std,gyroscope_std,gyroscope_uncalibrated_mean,accelerometer_max,linear_acceleration_mean,speed_mean,rotation_vector_mean,rotation_vector_max,accelerometer_min,magnetic_field_uncalibrated_min,sound_min,target
0,16170,Luca,0.001651,0.014626,0.000737,0.016221,9.849411,0.020978,0.000000,0.050413,0.056351,9.758895,51.199707,NaN,Still
1,15871,Luca,0.036326,1.775944,0.020290,0.039023,17.146631,0.879220,16.539349,0.999981,0.999999,7.707437,82.409890,89.065143,Car
2,16811,Luca,0.001525,0.011199,0.000713,0.016302,9.849262,0.034110,0.000000,0.610456,0.610456,9.804817,55.501802,NaN,Still
3,15831,Luca,0.036440,0.862553,0.010553,0.050759,12.304298,1.488361,17.739895,0.998112,0.998112,7.659674,95.664309,87.470377,Car
4,876,andrea,0.183202,0.504117,0.098819,0.265652,10.891645,1.658308,9.000000,0.322242,0.378193,8.965621,156.795909,89.770732,Car
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5888,14125,IvanHeibi,0.627658,3.373972,0.389385,0.476903,13.414389,4.934205,NaN,0.932682,0.941454,4.564648,56.502261,NaN,Walking
5889,19582,Pierpaolo,0.018259,0.665529,0.005964,0.018163,11.454213,0.544970,31.960054,0.295905,0.320622,8.768007,85.122017,NaN,Train
5890,16178,Luca,0.001803,0.017633,0.000904,0.016219,9.862587,0.020355,0.000000,0.051333,0.059438,9.772881,51.129444,NaN,Still
5891,10973,Damiano,0.003652,0.013485,0.001301,0.048900,9.874450,0.030764,1.090415,0.276348,0.276354,9.807347,270.328540,NaN,Still
